In [1]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from models import *
import time
import re, io, nltk, torch 
from nltk.corpus import stopwords
from numpy import linalg as LA
from numpy.linalg import norm
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
from sklearn.utils.class_weight import compute_class_weight
from gensim import utils, matutils  
from gensim.corpora.dictionary import Dictionary
from six import string_types, integer_types
from six.moves import zip, range
from numpy import linalg as LA
from sklearn.preprocessing import Normalizer
from scipy import stats
from gensim.utils import deprecated
from numpy import dot, float32 as REAL, memmap as np_memmap, \
    double, array, zeros, vstack, sqrt, newaxis, integer, \
    ndarray, sum as np_sum, prod, argmax
from collections import Counter
from scipy.spatial import distance
from numpy.random import binomial
from scipy.stats import bernoulli
import numpy as np


n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

if torch.cuda.is_available():     
    device = torch.device("cuda")
    print('There are',torch.cuda.device_count(),'GPU(s) available.')
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt


train =pd.read_csv("../Tesis_exp/Fine-Tuning/CSV_Harassment/train_format.csv")
val=pd.read_csv("../Tesis_exp/Fine-Tuning/CSV_Harassment/val_format.csv")
test=pd.read_csv("../Tesis_exp/Fine-Tuning/CSV_Harassment/test_format.csv")

dict_cl=dict()
dict_cl[0]="NonH"
dict_cl[1]="IndirectH"
dict_cl[2]="PhysicalH"
dict_cl[3]="SexualH"

#sdqc
d_lab=dict()
d_lab["NonH"]=0
d_lab["IndirectH"]=1
d_lab["PhysicalH"]=2
d_lab["SexualH"]=3

Using TensorFlow backend.
/home/casapanshop/anaconda2/envs/newpy3_tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/casapanshop/anaconda2/envs/newpy3_tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/casapanshop/anaconda2/envs/newpy3_tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1060 6GB


In [2]:
import re
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *

stemmer_sn = SnowballStemmer("english")
stemmer = PorterStemmer()

stoplist = stopwords.words("english")
lemmatizer=WordNetLemmatizer()

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

def cleaner(post): 
    #re.sub(r'([a-z])\1+', r'\1', "user i think that ' s all you loooooove ")
    s= re.sub(r"http\S+", " <html> ", post)   ##########elimina http    
    s= re.sub(r'#\w+ ?', " <hashtag> ", s) ########3
    s= re.sub(r'@\w+ ?', " <user> ", s)##############    
    s= s.lower()
    s=emoji_pattern.sub(r'', s)
    s=re.sub(r"'\b", " ' ", s)
    s=re.sub(r"\b'\b", " ' ", s)
    s=re.sub(r"“\b", " “ ", s)
    #“
    s=re.sub(r"\b’", " ’ ", s)
    s=re.sub(r"‘\b", " ‘ ", s)
    s=re.sub(r"\b’\b", " ‘ ", s)
    s = re.sub(r"-", " - ", s)
    s = re.sub(r"\(", " ", s)
    s = re.sub(r"\)", " ", s)
    s = re.sub(r"\?", " ? ", s)    
    s = re.sub(r"\/", " ", s)
    s = re.sub(r"' ", " ' ", s)
    s = re.sub(r" '", " ' ", s)
    s = re.sub(r"\!", " ! ", s)
    s=re.sub("[\.]+", " . ", s)
    s=re.sub("[\,]+", " , ", s)
    s=re.sub("[\;]+", " ; ", s)
    s=re.sub("[\:]+", " : ", s)
    s=re.sub('[\"]+', ' " ', s)
    s=re.sub(r'\b[0-9]\b', " <number> ",  s)
    s=re.sub(r'\b[0-9]*[0-9]\b', " <number> ",  s)    
    s=re.sub(r'\b”', ' " ', s)
    sl= list(s.split())
    sl_2=[]
    for wd in sl:
        try: 
            q=Word2Index_valid[wd]
            sl_2.append(wd)
        except:
            try: 
                if stemmer.stem(wd) in Word2Index_valid.keys():
                    sl_2.append(stemmer.stem(wd))
                elif lemmatizer.lemmatize(wd) in Word2Index_valid.keys():
                    sl_2.append(lemmatizer.lemmatize(wd))
                else:
                    sl_2.append(wd)
            except:
                sl_2.append(wd)
    sl=sl_2
    s=' '.join([word for word in sl])# if word not in stoplist])
    return s, sl

embeddings_index = {}
f = open(os.path.join("glove.twitter.27B.200d.txt"))#'glove.twitter.27B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Se encontraron %s terminos con sus vectores de embedding.' % len(embeddings_index))

Se encontraron 1193514 terminos con sus vectores de embedding.


In [3]:

word_index=dict()
j=1
for frase in train["Tweet"].values:
    frase=cleaner(frase)[-1]
    #print (frase)
    for term in frase:
        if term not in word_index.keys():
            word_index[term]=j
            j+=1

for frase in val["Tweet"].values:
    frase=cleaner(frase)[-1]
    #print (frase)
    for term in frase:
        if term not in word_index.keys():
            word_index[term]=j
            j+=1
            
for frase in test["Tweet"].values:
    frase=cleaner(frase)[-1]
    #print (frase)
    for term in frase:
        if term not in word_index.keys():
            word_index[term]=j
            j+=1
            
len(word_index.keys())

22107

In [4]:
not_found=[]
pos_tag_nf=[]
Word2Index={}
Word2Index_valid={}
idx_2_token={}
idx_2_token_valid={}

g_dim=200
emb_matrix = np.zeros((len(word_index.keys())+1, g_dim))  

k=1
for word, i in word_index.items():
    vector = embeddings_index.get(word)
    if vector is not None:
        # words sin match en Glove, serán vectores de ceros.
        emb_matrix[i] = vector
        Word2Index[word]=i
        Word2Index_valid[word]=k
        idx_2_token_valid[k]=word
        k+=1
        idx_2_token[i]=word
        
    else:
        Word2Index[word]=i
        idx_2_token[i]=word
        not_found.append(word)
        pos_tag_nf.append(nltk.pos_tag([word])[0][-1])
        
glove_matrix = np.zeros((k, g_dim))

for word, i in Word2Index_valid.items():
    vector = embeddings_index.get(word)    
    glove_matrix[i] = vector
    
glove_matrix.shape

(14642, 200)

In [5]:
#transf = Normalizer().fit(glove_matrix) 
#M_GLOVE_space=transf.transform(glove_matrix)
M_GLOVE_space=glove_matrix

def my_tokenizer(lista):
    encoded_sent=[]
    to_return_ide=[]
    for wd in lista:
        try:               
            to_return_ide.append(Word2Index_valid[wd])
            encoded_sent.append(wd)
        except: 
            continue   
           
    return to_return_ide, encoded_sent


def predict_data(trained, x_train, x_val, x_test, etiq, etiq_v, etiq_t, name_model):
    etiq = etiq.astype("int")
    trainPredict = trained.predict(x_train, batch_size=32)
    trainPredict=[np.argmax(pred) for pred in trainPredict]
    acc= accuracy_score(etiq, trainPredict)  
    f1=f1_score(etiq, trainPredict, average=None)  # labels=np.unique(trainPredict)
    f1_ma=f1_score(etiq, trainPredict, average='macro')  # labels=np.unique(trainPredict)
    #matriz=normalize(confusion_matrix(etiq, trainPredict))
    print ("")
    print ("Accuracy sobre Train", name_model, ":",acc)  
    print ("F1-score None sobre Train", name_model, ":",f1)
    print ("F1-score macro sobre Train", name_model, ":",f1_ma)
    ########################################
    trainPredict = trained.predict(x_val, batch_size=32)
    trainPredict=[np.argmax(pred) for pred in trainPredict]
    acc= accuracy_score(etiq_v, trainPredict)  
    f1=f1_score(etiq_v, trainPredict, average=None)  # labels=np.unique(trainPredict)
    f1_ma=f1_score(etiq_v, trainPredict, average='macro')  # labels=np.unique(trainPredict)
    #matriz=normalize(confusion_matrix(etiq_v, trainPredict))
    print ("")
    print ("Accuracy sobre Val", name_model, ":",acc)  
    print ("F1-score None sobre Val", name_model, ":",f1)
    print ("F1-score macro sobre Val", name_model, ":",f1_ma)
    ########################################
    trainPredict = trained.predict(x_test, batch_size=32)
    trainPredict=[np.argmax(pred) for pred in trainPredict]
    acc_t= accuracy_score(etiq_t, trainPredict)  
    f1_t=f1_score(etiq_t, trainPredict, average=None)  # labels=np.unique(trainPredict)
    f1_ma_t=f1_score(etiq_t, trainPredict, average='macro')  # labels=np.unique(trainPredict)
    matriz_t=normalize(confusion_matrix(etiq_t, trainPredict))
    print ("")
    print ("Accuracy sobre Test", name_model, ":",acc_t)  
    print ("F1-score None sobre Test", name_model, ":",f1_t)
    print ("F1-score macro sobre Test", name_model, ":",f1_ma_t)
    
    return f1_ma_t, f1_t, acc_t, matriz_t

In [6]:
from models import *

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
class Dummy_Embeddings(nn.Module):
    def __init__(self, d_model, dummy_vectors): 
        super(Dummy_Embeddings, self).__init__()
        aux = torch.from_numpy(dummy_vectors)#, dtype='float32'))
        self.index2dummy = nn.Embedding(aux.size()[0], d_model)
        self.index2dummy.weigth=nn.Parameter(aux)
        self.index2dummy.weigth.requires_grad=False
        self.d_model = d_model
        
    def forward(self, x):
        aux=x.numpy()
        #print ("aux original", aux)
        #print ("aux restado...",aux-np.ones(aux.shape))
        new_x= aux #- np.ones(aux.shape)
        new_x= torch.from_numpy(new_x)
        return self.index2dummy(new_x.long()) * math.sqrt(self.d_model) #debiese retornar matriz de batch_size x [ind_tw, k1,k2,k3,k4,k5,k6] (si son 6 modelos)
    
def match(objetos,ejemplo):
    i=0
    for obj in objetos:
        if obj==ejemplo:
            return i
        i+=1
        
def plot_confusion_matrix(cm, target_names, title='Confusion matrix (f1-score)',cmap=None, normalize=True):    
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
    if cmap is None:
        cmap = plt.get_cmap('Blues')
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=1)
    plt.title(title)
    plt.colorbar()
    
    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)
    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()
    
MAX_LEN = 33

print ("Conteo por stance val\n", Counter(val['Label']))
print ("Conteo por stance test\n", Counter(test['Label']))


Conteo por stance val
 Counter({'NonH': 1493, 'SexualH': 525, 'IndirectH': 71, 'PhysicalH': 36})
Conteo por stance test
 Counter({'NonH': 1601, 'SexualH': 340, 'IndirectH': 106, 'PhysicalH': 76})


In [7]:
ids_train=train['tweet_id']
tws_train=train['Tweet']
labels_train=[d_lab[lb] for lb in train['Label']]
#class_weights= compute_class_weight('balanced', np.unique(labels_train), labels_train)
class_weights= [1.0, 1.0, 1.0, 1.0]
class_weights=np.asarray(class_weights)
print ("Pesos de clases:", class_weights)
#print ("Pesos de clases:", class_weights)

etiq=labels_train.copy()
etiq=np.asarray(etiq)
sentences = tws_train
n_labels = np.array(etiq)
y_train=to_categorical(n_labels,num_classes=4)


input_ids=[]
for sent in sentences:
    encoded_sent, sent_valid = my_tokenizer(cleaner(sent)[1])  
    input_ids.append(encoded_sent) 
    
input_ids = pad_sequences(input_ids,maxlen=MAX_LEN,dtype="long",value=0, truncating="post", padding="post")
shape=np.asarray(input_ids).shape
x_train=np.zeros((shape[0], shape[1], 200))
j=0
for in_id in input_ids:
    x_train[j]=M_GLOVE_space[in_id]
    j+=1

Pesos de clases: [1. 1. 1. 1.]


In [8]:
ids_val=val['tweet_id']
tws_val=val['Tweet']
labels_val=[d_lab[lb] for lb in val['Label']]

etiq_v=labels_val.copy()
etiq_v=np.asarray(etiq_v)
sentences_val = tws_val
n_labels_val = np.array(etiq_v)
y_val=to_categorical(n_labels_val,num_classes=4)

input_ids_val=[]
for sent in sentences_val:
    encoded_sent, sent_valid =my_tokenizer(cleaner(sent)[1])  
    input_ids_val.append(encoded_sent)

ids_test=test['tweet_id']
tws_test=test['Tweet']
labels_test=[d_lab[lb] for lb in test['Label']]

etiq_t=labels_test.copy()
etiq_t=np.asarray(etiq_t)
sentences_test = tws_test
n_labels_test = np.array(etiq_t)
y_test=to_categorical(n_labels_test,num_classes=4)

input_ids_test=[]
for sent in sentences_test:
    encoded_sent, sent_valid = my_tokenizer(cleaner(sent)[1])
    input_ids_test.append(encoded_sent)
    
input_ids_val = pad_sequences(input_ids_val,maxlen=MAX_LEN,dtype="long",value=0, truncating="post", padding="post")
input_ids_test = pad_sequences(input_ids_test,maxlen=MAX_LEN,dtype="long",value=0, truncating="post", padding="post")


shape_val=np.asarray(input_ids_val).shape
x_val=np.zeros((shape_val[0], shape_val[1], 200))
i=0
for in_id in input_ids_val:
    x_val[i]=M_GLOVE_space[in_id]
    i+=1
    
shape_test=np.asarray(input_ids_test).shape
x_test=np.zeros((shape_test[0], shape_test[1], 200))
i=0
for in_id in input_ids_test:
    x_test[i]=M_GLOVE_space[in_id]
    i+=1

x_test.shape

(2123, 33, 200)

In [9]:
def predecir_modelos(bs,predichos_all):  #best fit
    temp=predichos_all.copy()
    final=[np.argmax(pred) for pred in temp]  
    confianzas=[temp[i][final[i]] for i in range(len(final))]
    predichos_new=[]
    for i in range(len(temp)):
        indice=final[i]%4
        predichos_new.append(int(indice))
    return predichos_new,final,confianzas    

def predecir_modelos_norm(bs,predichos_all):  #normalizado
    temp=predichos_all.copy()
    final=[]
    confianzas=[]
    for pred in temp:   
        aux=np.ones(4)
        for i in range(1,6): #for machine pred
            sub=pred[(4*(i-1)):(4*i)]
            aux=aux*np.asarray(sub)
            
        aux=aux/np.sum(aux)
        final.append(np.argmax(aux))
        confianzas.append(aux[np.argmax(aux)])  
           
    predichos_new=[]
    for i in range(len(temp)):
        predichos_new.append(final[i])
    return predichos_new,final,confianzas
    
def predecir_modelos_average(bs,predichos_all): #average
    temp=predichos_all.copy()
    final=[]
    confianzas=[]
    for pred in temp:
        aux=np.zeros(4)
        for i in range(1,6):
            sub=pred[(4*(i-1)):(4*i)]
            aux=aux+np.asarray(sub)
            
        aux=aux/5.0 #dividido en el total de machines
        final.append(np.argmax(aux))
        confianzas.append(aux[np.argmax(aux)])  
           
    predichos_new=[]
    for i in range(len(temp)):
        predichos_new.append(final[i])
    return predichos_new,final,confianzas


In [10]:
del embeddings_index
del emb_matrix

In [11]:
class_weights

array([1., 1., 1., 1.])

### 0.0 sin pesos

In [13]:
prob=0.0
path='../new_glove_augmented/harassment_baselines/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

for j in range(1,6):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j)+'----------------')
    cnn1= load_model(path+str(prob)+'/cnn1_'+str(j)+'-exec_15.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'/cnn2_'+str(j)+'-exec_15.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'/rnn1_'+str(j)+'-exec_8.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'/rnn2_'+str(j)+'-exec_15.h5') 
    rnn3= load_model(path+str(prob)+'/rnn3_'+str(j)+'-exec_8.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    """print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)"""
    
    print ("Agregando predicciones Test set")    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    """print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)"""
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1_no)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score SDQC:",f1_no)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1_no)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)



---------------- EJECUCIÓN #1----------------
Agregando predicciones Test set
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.8412623645784267
F1-macro: 0.42037599767404776
F1-score SDQC: [0.90909091 0.03703704 0.         0.73537604]

Committee Norm
Accuracy: 0.8586905322656618
F1-macro: 0.42520389989438656
F1-score SDQC: [0.92075695 0.         0.         0.78005865]

Committee Voting
Accuracy: 0.8652849740932642
F1-macro: 0.4306337411217853
F1-score SDQC: [0.92493377 0.         0.         0.7976012 ]

---------------- EJECUCIÓN #2----------------
Agregando predicciones Test set
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.855864342910975
F1-macro: 0.4303700262206065
F1-score SDQC: [0.91947291 0.01869159 0.02597403 0.75734158]

Committee Norm
Accuracy: 0.8615167216203485
F1-macro: 0.42566195994739064
F1-score SDQC: [0.92298682 0.         0.         0.77966102]

Committee Voting
Accuracy: 0.8634008478568064
F1-macro: 0.42773115621476276
F1-score SDQC: [0.9238

In [14]:
#### 15-8 epochs

m=5.0

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/m))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Best comité:" ,temp/m)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/m))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Norm comité:" ,temp/m)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/m))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Voting comité:" ,temp/m)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/m))


[TEST] Acc Best comité: 0.8507772020725389
[TEST] F1-score NIPS Best comité: [0.91648735 0.03777329 0.03033263 0.75043578]
[TEST] F1 macro Best comité: 0.43375726259340214
-----------
[TEST] Acc Norm comité: 0.861987753179463
[TEST] F1-score NIPS Norm comité: [0.92317723 0.         0.         0.78206736]
[TEST] F1 macro Norm comité: 0.4263111459847867
-----------
[TEST] Acc Voting comité: 0.86547338671691
[TEST] F1-score NIPS Voting comité: [0.92522014 0.         0.         0.79263126]
[TEST] F1 macro Voting comité: 0.4294628497052077


In [13]:
m=5.0

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/m))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Best comité:" ,temp/m)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/m))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Norm comité:" ,temp/m)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/m))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Voting comité:" ,temp/m)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/m))


[TEST] Acc Best comité: 0.8468205369759774
[TEST] F1-score NIPS Best comité: [0.91406858 0.04135088 0.05606891 0.75117568]
[TEST] F1 macro Best comité: 0.4406660129178516
-----------
[TEST] Acc Norm comité: 0.8606688648139427
[TEST] F1-score NIPS Norm comité: [0.92220497 0.         0.         0.78497927]
[TEST] F1 macro Norm comité: 0.42679605912453833
-----------
[TEST] Acc Voting comité: 0.8653791804050872
[TEST] F1-score NIPS Voting comité: [0.92485427 0.         0.00512821 0.79685099]
[TEST] F1 macro Voting comité: 0.4317083656478837


### 0.0 cw

In [16]:
prob=0.0
path='../new_glove_augmented/harassment_baselines/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

for j in range(1,6):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j)+'----------------')
    cnn1= load_model(path+str(prob)+'_cw/cnn1_'+str(j)+'-exec_15e.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'_cw/cnn2_'+str(j)+'-exec_15e.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'_cw/rnn1_'+str(j)+'-exec_8e.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'_cw/rnn2_'+str(j)+'-exec_15e.h5') 
    rnn3= load_model(path+str(prob)+'_cw/rnn3_'+str(j)+'-exec_8e.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    """ print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    """
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    """
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score NIPS:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)"""
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score NIPS:",f1_no)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score NIPS:",f1_no)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score NIPS:",f1_no)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)



---------------- EJECUCIÓN #1----------------
Agregando predicciones Test set rnn3
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.8516250588789449
F1-macro: 0.46930185603667507
F1-score NIPS: [0.91748983 0.11290323 0.11494253 0.73187184]

Committee Norm
Accuracy: 0.8601036269430051
F1-macro: 0.4582785508143068
F1-score NIPS: [0.92001155 0.07079646 0.07228916 0.77001704]

Committee Voting
Accuracy: 0.8591615638247763
F1-macro: 0.483824932617313
F1-score NIPS: [0.91868512 0.12173913 0.13793103 0.75694444]

---------------- EJECUCIÓN #2----------------
Agregando predicciones Test set rnn3
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.8327837965143665
F1-macro: 0.5037770191998525
F1-score NIPS: [0.91159251 0.26506024 0.18644068 0.65201465]

Committee Norm
Accuracy: 0.8610456900612341
F1-macro: 0.5288769375844222
F1-score NIPS: [0.92186589 0.21374046 0.22222222 0.75767918]

Committee Voting
Accuracy: 0.8497409326424871
F1-macro: 0.5243393354734585
F1-score NIPS: 

In [17]:
print ("Con 15 y 8 epochs...\n")
m=5.0

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/m))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Best comité:" ,temp/m)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/m))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Norm comité:" ,temp/m)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/m))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Voting comité:" ,temp/m)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/m))


Con 15 y 8 epochs...

[TEST] Acc Best comité: 0.84625529910504
[TEST] F1-score NIPS Best comité: [0.91536736 0.16600064 0.15973436 0.71938417]
[TEST] F1 macro Best comité: 0.49012163234043116
-----------
[TEST] Acc Norm comité: 0.8642487046632125
[TEST] F1-score NIPS Norm comité: [0.92353037 0.08776435 0.13736381 0.77995598]
[TEST] F1 macro Norm comité: 0.4821536292725838
-----------
[TEST] Acc Voting comité: 0.8624587847385776
[TEST] F1-score NIPS Voting comité: [0.92302732 0.14678328 0.17337898 0.76462381]
[TEST] F1 macro Voting comité: 0.5019533485048798


In [15]:
m=5.0

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/m))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Best comité:" ,temp/m)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/m))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Norm comité:" ,temp/m)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/m))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Voting comité:" ,temp/m)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/m))


[TEST] Acc Best comité: 0.8424870466321244
[TEST] F1-score NIPS Best comité: [0.91344209 0.15978576 0.1698437  0.73865242]
[TEST] F1 macro Best comité: 0.4954309951957939
-----------
[TEST] Acc Norm comité: 0.8696184644371172
[TEST] F1-score NIPS Norm comité: [0.92761455 0.14946848 0.11116994 0.80520837]
[TEST] F1 macro Norm comité: 0.49836533854148113
-----------
[TEST] Acc Voting comité: 0.8700894959962318
[TEST] F1-score NIPS Voting comité: [0.92713953 0.22937507 0.18281202 0.81110401]
[TEST] F1 macro Voting comité: 0.5376076561310092


### 0.15

In [18]:
prob=0.15
path='../new_glove_augmented/harassment_baselines/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

for j in range(1,6):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j)+'----------------')
    cnn1= load_model(path+str(prob)+'/cnn1_'+str(j)+'-exec_app1_Top_1_7e.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'/cnn2_'+str(j)+'-exec_app1_Top_1_7e.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'/rnn1_'+str(j)+'-exec_app1_Top_1_4e.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'/rnn2_'+str(j)+'-exec_app1_Top_1_7e.h5') 
    rnn3= load_model(path+str(prob)+'/rnn3_'+str(j)+'-exec_app1_Top_1_4e.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
        
    print ("Agregando predicciones Test set")    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
       
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score NIPS:",f1_no)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score NIPS:",f1_no)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score NIPS:",f1_no)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)



---------------- EJECUCIÓN #1----------------
Agregando predicciones Test set
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.8186528497409327
F1-macro: 0.5266927336555304
F1-score NIPS: [0.89801155 0.23589744 0.24074074 0.73212121]

Committee Norm
Accuracy: 0.8506829957607159
F1-macro: 0.4730888141701334
F1-score NIPS: [0.92008639 0.10606061 0.09756098 0.76864728]

Committee Voting
Accuracy: 0.8539802166745172
F1-macro: 0.48171642863397834
F1-score NIPS: [0.92236599 0.13333333 0.0952381  0.7759283 ]

---------------- EJECUCIÓN #2----------------
Agregando predicciones Test set
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.8459726801695714
F1-macro: 0.4766334958570899
F1-score NIPS: [0.91756053 0.1369863  0.08791209 0.76407507]

Committee Norm
Accuracy: 0.8662270372114932
F1-macro: 0.44744623774865466
F1-score NIPS: [0.92831325 0.03669725 0.02531646 0.79945799]

Committee Voting
Accuracy: 0.8648139425341498
F1-macro: 0.4473615881127515
F1-score NIPS: [0.92678

In [19]:
print ("con 7 y 4 epochs... \n")

m=5.0

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/m))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Best comité:" ,temp/m)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/m))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Norm comité:" ,temp/m)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/m))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Voting comité:" ,temp/m)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/m))


con 7 y 4 epochs... 

[TEST] Acc Best comité: 0.8425812529439473
[TEST] F1-score NIPS Best comité: [0.91347946 0.15904215 0.19023584 0.75672402]
[TEST] F1 macro Best comité: 0.5048703683707564
-----------
[TEST] Acc Norm comité: 0.8604804521902967
[TEST] F1-score NIPS Norm comité: [0.92448436 0.06371117 0.0541202  0.78522804]
[TEST] F1 macro Norm comité: 0.45688594464368887
-----------
[TEST] Acc Voting comité: 0.8624587847385775
[TEST] F1-score NIPS Voting comité: [0.92551085 0.07533428 0.08480822 0.79027801]
[TEST] F1 macro Voting comité: 0.4689828385203003


In [15]:
m=5.0

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/m))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Best comité:" ,temp/m)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/m))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Norm comité:" ,temp/m)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/m))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Voting comité:" ,temp/m)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/m))


[TEST] Acc Best comité: 0.8399434762129062
[TEST] F1-score NIPS Best comité: [0.91021617 0.09467849 0.14992861 0.74687985]
[TEST] F1 macro Best comité: 0.4754257831049076
-----------
[TEST] Acc Norm comité: 0.856806406029204
[TEST] F1-score NIPS Norm comité: [0.92056639 0.04679765 0.03509661 0.77534944]
[TEST] F1 macro Norm comité: 0.4444525226332764
-----------
[TEST] Acc Voting comité: 0.8598210080075365
[TEST] F1-score NIPS Voting comité: [0.92231087 0.05602408 0.0385772  0.7842442 ]
[TEST] F1 macro Voting comité: 0.4502890872724369


### 0.5

In [20]:
prob=0.5
path='../new_glove_augmented/harassment_baselines/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

for j in range(1,6):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j)+'----------------')
    cnn1= load_model(path+str(prob)+'/cnn1_'+str(j)+'-exec_app1_Top_1_7e.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'/cnn2_'+str(j)+'-exec_app1_Top_1_7e.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'/rnn1_'+str(j)+'-exec_app1_Top_1_4e.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'/rnn2_'+str(j)+'-exec_app1_Top_1_7e.h5') 
    rnn3= load_model(path+str(prob)+'/rnn3_'+str(j)+'-exec_app1_Top_1_4e.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    
    print ("Agregando predicciones Test set")    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
        
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score NIPS:",f1_no)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score NIPS:",f1_no)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score NIPS:",f1_no)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)



---------------- EJECUCIÓN #1----------------
Agregando predicciones Test set
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.795572303344324
F1-macro: 0.45053968785636234
F1-score NIPS: [0.88336026 0.12820513 0.11363636 0.676957  ]

Committee Norm
Accuracy: 0.8337258596325954
F1-macro: 0.44327893971762566
F1-score NIPS: [0.90897717 0.08064516 0.05063291 0.73286052]

Committee Voting
Accuracy: 0.8356099858690532
F1-macro: 0.4596241741530389
F1-score NIPS: [0.90994371 0.09230769 0.0952381  0.74100719]

---------------- EJECUCIÓN #2----------------
Agregando predicciones Test set
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.8097032501177579
F1-macro: 0.48226138971559585
F1-score NIPS: [0.89225908 0.01652893 0.32653061 0.69372694]

Committee Norm
Accuracy: 0.8276024493641074
F1-macro: 0.4660125270317209
F1-score NIPS: [0.90328638 0.         0.24       0.72076372]

Committee Voting
Accuracy: 0.8341968911917098
F1-macro: 0.49217972383261177
F1-score NIPS: [0.9069

In [21]:
print ("con 7 y 4 epochs...\n")

m=5.0

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/m))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Best comité:" ,temp/m)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/m))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Norm comité:" ,temp/m)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/m))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Voting comité:" ,temp/m)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/m))


con 7 y 4 epochs...

[TEST] Acc Best comité: 0.824493641073952
[TEST] F1-score NIPS Best comité: [0.90167547 0.10515991 0.2060048  0.72214993]
[TEST] F1 macro Best comité: 0.4837475270226802
-----------
[TEST] Acc Norm comité: 0.8460668864813943
[TEST] F1-score NIPS Norm comité: [0.9142685  0.0441811  0.14941438 0.75770549]
[TEST] F1 macro Norm comité: 0.4663923664294508
-----------
[TEST] Acc Voting comité: 0.8469147432878004
[TEST] F1-score NIPS Voting comité: [0.91468421 0.06367413 0.18606393 0.7618335 ]
[TEST] F1 macro Voting comité: 0.4815639432692074


In [17]:
m=5.0

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/m))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Best comité:" ,temp/m)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/m))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Norm comité:" ,temp/m)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/m))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Voting comité:" ,temp/m)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/m))


[TEST] Acc Best comité: 0.8071596796985397
[TEST] F1-score NIPS Best comité: [0.89049212 0.11073239 0.17162255 0.70024128]
[TEST] F1 macro Best comité: 0.4682720840623157
-----------
[TEST] Acc Norm comité: 0.8355157795572303
[TEST] F1-score NIPS Norm comité: [0.90888367 0.05757599 0.11769108 0.73535642]
[TEST] F1 macro Norm comité: 0.4548767893352969
-----------
[TEST] Acc Voting comité: 0.8416391898257184
[TEST] F1-score NIPS Voting comité: [0.91301854 0.06599851 0.12760449 0.7500492 ]
[TEST] F1 macro Voting comité: 0.4641676870158177


### 0.85

In [22]:
prob=0.85
path='../new_glove_augmented/harassment_baselines/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

for j in range(1,6):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j)+'----------------')
    cnn1= load_model(path+str(prob)+'/cnn1_'+str(j)+'-exec_app1_Top_1_7e.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'/cnn2_'+str(j)+'-exec_app1_Top_1_7e.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'/rnn1_'+str(j)+'-exec_app1_Top_1_4e.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'/rnn2_'+str(j)+'-exec_app1_Top_1_7e.h5') 
    rnn3= load_model(path+str(prob)+'/rnn3_'+str(j)+'-exec_app1_Top_1_4e.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    
    print ("Agregando predicciones Test set")    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
        
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score NIPS:",f1_no)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score NIPS:",f1_no)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score NIPS:",f1_no)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)



---------------- EJECUCIÓN #1----------------
Agregando predicciones Test set
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.8365520489872822
F1-macro: 0.47205939195464786
F1-score NIPS: [0.90750226 0.11023622 0.13592233 0.73457676]

Committee Norm
Accuracy: 0.8469147432878003
F1-macro: 0.463786817741373
F1-score NIPS: [0.91399461 0.05042017 0.14285714 0.74787535]

Committee Voting
Accuracy: 0.8483278379651437
F1-macro: 0.45632648963694733
F1-score NIPS: [0.91531532 0.08264463 0.07594937 0.75139665]

---------------- EJECUCIÓN #2----------------
Agregando predicciones Test set
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.7739048516250588
F1-macro: 0.5014794206982005
F1-score NIPS: [0.86315087 0.24892704 0.20869565 0.68514412]

Committee Norm
Accuracy: 0.8134715025906736
F1-macro: 0.48626285927733887
F1-score NIPS: [0.89252486 0.16759777 0.15384615 0.73108265]

Committee Voting
Accuracy: 0.8186528497409327
F1-macro: 0.49548382246161643
F1-score NIPS: [0.8967

In [23]:
print ("con 7 y 4 epochs...\n")

m=5.0

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/m))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Best comité:" ,temp/m)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/m))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Norm comité:" ,temp/m)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/m))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Voting comité:" ,temp/m)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/m))


con 7 y 4 epochs...

[TEST] Acc Best comité: 0.8187470560527554
[TEST] F1-score NIPS Best comité: [0.89598594 0.15257961 0.17722427 0.7252228 ]
[TEST] F1 macro Best comité: 0.4877531559120173
-----------
[TEST] Acc Norm comité: 0.8408855393311352
[TEST] F1-score NIPS Norm comité: [0.9111881  0.08876562 0.0778871  0.75274483]
[TEST] F1 macro Norm comité: 0.4576464104777421
-----------
[TEST] Acc Voting comité: 0.8428638718794159
[TEST] F1-score NIPS Voting comité: [0.91259835 0.1023185  0.07730133 0.75820216]
[TEST] F1 macro Voting comité: 0.4626050863142679


In [19]:
m=5.0

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/m))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Best comité:" ,temp/m)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/m))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Norm comité:" ,temp/m)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/m))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Voting comité:" ,temp/m)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/m))


[TEST] Acc Best comité: 0.81921808761187
[TEST] F1-score NIPS Best comité: [0.89791142 0.08312651 0.08559012 0.70417871]
[TEST] F1 macro Best comité: 0.4427016889619424
-----------
[TEST] Acc Norm comité: 0.8373999057936882
[TEST] F1-score NIPS Norm comité: [0.9094091  0.040043   0.025      0.73757739]
[TEST] F1 macro Norm comité: 0.4280073730785264
-----------
[TEST] Acc Voting comité: 0.8395666509656147
[TEST] F1-score NIPS Voting comité: [0.91089089 0.0426081  0.03826736 0.74392055]
[TEST] F1 macro Voting comité: 0.43392172396770323
